In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset
import pandas as pd
import random

random.seed(42)

swda = load_dataset("cgpotts/swda", split="train")

df = pd.DataFrame(swda)
df = df.sort_values(by=["conversation_no", "transcript_index"])

male_pairs = []
female_pairs = []

for conv_id, group in df.groupby("conversation_no"):
    group = group.sort_values("transcript_index")
    utterances = group["text"].tolist()
    responder_genders = group["from_caller_sex"].tolist()
    for i in range(len(utterances) - 1):
        prompt = utterances[i]
        response = utterances[i + 1]
        responder_gender = responder_genders[i + 1]
        formatted = f"Prompt: {prompt} Response: {response}"
        if responder_gender == "MALE":
            male_pairs.append(formatted)
        elif responder_gender == "FEMALE":
            female_pairs.append(formatted)

# Balance the datasets
min_count = min(len(male_pairs), len(female_pairs))
male_balanced = random.sample(male_pairs, min_count) if len(male_pairs) > min_count else male_pairs
female_balanced = random.sample(female_pairs, min_count) if len(female_pairs) > min_count else female_pairs

# Save balanced datasets
with open("swda_male_balanced.txt", "w", encoding="utf-8") as f:
    for pair in male_balanced:
        f.write(pair + "\n")

with open("swda_female_balanced.txt", "w", encoding="utf-8") as f:
    for pair in female_balanced:
        f.write(pair + "\n")

print(f"Balanced datasets saved: {len(male_balanced)} male and {len(female_balanced)} female pairs.")


The repository for cgpotts/swda contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cgpotts/swda.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/213543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56729 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4514 [00:00<?, ? examples/s]

Balanced datasets saved: 84210 male and 84210 female pairs.


In [ ]:
df


,swda_filename,ptb_basename,conversation_no,transcript_index,act_tag,damsl_act_tag,caller,utterance_index,subutterance_index,text,...,from_caller,from_caller_sex,from_caller_education,from_caller_birth_year,from_caller_dialect_area,to_caller,to_caller_sex,to_caller_education,to_caller_birth_year,to_caller_dialect_area
65878,sw03utt/sw_0384_2005.utt,2/sw2005,2005,0,115,26,A,1,1,Okay. /,...,1169,MALE,2,1961,NORTH MIDLAND,1133,MALE,2,1935,SOUTH MIDLAND
65879,sw03utt/sw_0384_2005.utt,2/sw2005,2005,1,63,1,A,1,2,"{F Uh, } first, {F um, } I need to know, {F uh...",...,1169,MALE,2,1961,NORTH MIDLAND,1133,MALE,2,1935,SOUTH MIDLAND
65880,sw03utt/sw_0384_2005.utt,2/sw2005,2005,2,132,12,B,2,1,"{D Well, } of course, [ it's, + {D you know, }...",...,1169,MALE,2,1961,NORTH MIDLAND,1133,MALE,2,1935,SOUTH MIDLAND
65881,sw03utt/sw_0384_2005.utt,2/sw2005,2005,3,18,11,A,3,1,Yes. /,...,1169,MALE,2,1961,NORTH MIDLAND,1133,MALE,2,1935,SOUTH MIDLAND
65882,sw03utt/sw_0384_2005.utt,2/sw2005,2005,4,18,11,A,3,2,Yeah. /,...,1169,MALE,2,1961,NORTH MIDLAND,1133,MALE,2,1935,SOUTH MIDLAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135572,sw07utt/sw_0733_4940.utt,4/sw4940,4940,188,125,4,A,113,1,{C Cause } I haven't been there probably ... -/,...,1423,FEMALE,2,1960,WESTERN,1662,MALE,2,1959,WESTERN
135573,sw07utt/sw_0733_4940.utt,4/sw4940,4940,189,66,2,B,114,1,"{D Well, } {D you know, } [ yo-, + ] going ove...",...,1423,FEMALE,2,1960,WESTERN,1662,MALE,2,1959,WESTERN
135574,sw07utt/sw_0733_4940.utt,4/sw4940,4940,190,194,21,A,115,1,Yeah. /,...,1423,FEMALE,2,1960,WESTERN,1662,MALE,2,1959,WESTERN
135575,sw07utt/sw_0733_4940.utt,4/sw4940,4940,191,125,4,B,116,1,"You can see that horrible, horrible brown haze. /",...,1423,FEMALE,2,1960,WESTERN,1662,MALE,2,1959,WESTERN


In [ ]:
import re

def clean_text(text):
    text = re.sub(r"\{[^}]*\}", '', text)
    text = re.sub(r"\[[^]]*\]", '', text)
    text = re.sub(r"[/+–]", ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[#*\-]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Clean residual punctuation and whitespace
    text = re.sub(r'\s+', ' ', text).strip()  # Single spaces
    text = re.sub(r'([,.!?])\1+', r'\1', text)  # Remove repeated punctuation
    return text.strip(' ,.')  # Trim leading/trailing punctuation

# Test with your example
input_text = "Prompt: {D Well, } of course, [ it's, + {D you know, } it's ] one of... / Response: Yes.  /"
for text in male_balanced:
  cmt = clean_text(text)


In [ ]:
cmt

'Prompt: Because, treated wood will also chew them up. Response: Treated wood can really make you sick too'

In [ ]:
def clean_file(input_path, output_path):
    with open(input_path, 'r') as f_in, open(output_path, 'w') as f_out:
        for line in f_in:
            cleaned = clean_text(line)
            f_out.write(cleaned + '\n')

clean_file("swda_male_balanced.txt", "swda_male_clean.txt")
clean_file("swda_female_balanced.txt", "swda_female_clean.txt")



In [ ]:
for gender in ["male", "female"]:
    with open(f"swda_{gender}_clean.txt", "r") as f:
        cleaned = [clean_text(line) for line in f]
    with open(f"swda_{gender}_clean_again.txt", "w") as f:
        f.write("\n".join(cleaned))


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

def train_model(dataset_path, output_dir):

    dataset = load_dataset("text", data_files=dataset_path)["train"]
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, max_length=128)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # For causal language modeling
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_strategy="no",
        logging_dir=f"{output_dir}/logs",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    trainer.train()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

train_model("swda_male_clean_again.txt", "gpt2-male")
train_model("swda_female_clean_again.txt", "gpt2-female")


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.013700
1000,2.890100
1500,2.846900
2000,2.807000
2500,2.813400
3000,2.781800
3500,2.812700
4000,2.773100
4500,2.755700
5000,2.735800


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/84210 [00:00<?, ? examples/s]

Step,Training Loss
500,2.955600
1000,2.791100
1500,2.746300
2000,2.731800
2500,2.702800
3000,2.695400
3500,2.711600
4000,2.665200
4500,2.709900
5000,2.666000


In [ ]:
from transformers import pipeline
male_model = GPT2LMHeadModel.from_pretrained("gpt2-male")
male_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-male")
male_generator = pipeline("text-generation", model=male_model, tokenizer=male_tokenizer)

female_model = GPT2LMHeadModel.from_pretrained("gpt2-female")
female_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-female")
female_generator = pipeline("text-generation", model=female_model, tokenizer=female_tokenizer)


Device set to use cuda:0
Device set to use cuda:0


In [ ]:
prompt = "Prompt: What do you enjoy doing in your free time? Response: "

male_output = male_generator(prompt, max_length=60, do_sample=True, top_k=50, top_p=0.95, temperature=0.8, truncation = True)
print("Male model:", male_output[0]["generated_text"])

female_output = female_generator(prompt, max_length=60, do_sample=True, top_k=50, top_p=0.95, temperature=0.8, truncation = True)
print("Female model:", female_output[0]["generated_text"])


Male model: Prompt: What do you enjoy doing in your free time? Response: What do you do with your spare time? What are you studying? > I'm just studying for an Master's in Psychology right now, though, right now, because I
Female model: Prompt: What do you enjoy doing in your free time? Response: I like to play tennis and play racquetball in my free time, although, I don't usually enjoy playing tennis in my free time either. Because I love to enjoy


In [ ]:
import shutil
shutil.make_archive('gpt2-female', 'zip', 'gpt2-female')


'/content/gpt2-female.zip'

In [ ]:
from google.colab import files
files.download('gpt2-female.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
shutil.make_archive('gpt2-male', 'zip', 'gpt2-male')


'/content/gpt2-male.zip'

In [ ]:
from google.colab import files
files.download('gpt2-male.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:

prompts = [
    "What do you enjoy doing in your free time?",
    "Can you describe your perfect weekend?",
    "What’s the most interesting book you’ve read recently?",
    "Do you have any hobbies or activities you’re passionate about?",
    "Where is your favorite place you’ve ever visited?",
    "What kind of music do you like to listen to?",
    "If you could learn any new skill, what would it be?",
    "What’s your favorite meal to cook or eat?",
    "Have you watched any good movies or TV shows lately?",
    "Do you have any pets? Tell me about them.",
    "What’s something new you’ve tried recently?",
    "How do you like to relax after a long day?",
    "What’s a goal you’re working towards right now?",
    "What’s your favorite way to stay active or exercise?",
    "If you could travel anywhere in the world, where would you go?",
    "What’s a memorable event from your childhood?",
    "Do you prefer mornings or evenings, and why?",
    "What’s your favorite season of the year?",
    "Are you a coffee or tea person?",
    "What’s a small thing that made you happy recently?",
    "What’s your favorite thing about your hometown?",
    "Do you enjoy cooking or baking? What’s your specialty?",
    "What’s a fun fact about yourself?",
    "If you could meet any historical figure, who would it be?",
    "What’s your favorite way to spend time with friends?",
    "Do you like to plan ahead or be spontaneous?",
    "What’s your favorite holiday or celebration?",
    "What’s something you’re grateful for today?",
    "What’s one thing you’d love to learn more about?",
    "What’s your favorite way to unwind on the weekend?"
]

results = []

for prompt_text in prompts:
    prompt = f"Prompt: {prompt_text} Response: "
    for i in range(5):  # 5 responses per prompt
        male_output = male_generator(
            prompt, max_length=60, do_sample=True, top_k=50, top_p=0.95, temperature=0.8, truncation=True,
        )[0]["generated_text"]
        female_output = female_generator(
            prompt, max_length=60, do_sample=True, top_k=50, top_p=0.95, temperature=0.8, truncation=True
        )[0]["generated_text"]
        results.append({
            "prompt": prompt_text,
            "model": "male",
            "response": male_output
        })
        results.append({
            "prompt": prompt_text,
            "model": "female",
            "response": female_output
        })

df = pd.DataFrame(results)
df.to_csv("gpt2_gendered_responses.csv", index=False)

print(df.head())


                                       prompt   model  \
0  What do you enjoy doing in your free time?    male   
1  What do you enjoy doing in your free time?  female   
2  What do you enjoy doing in your free time?    male   
3  What do you enjoy doing in your free time?  female   
4  What do you enjoy doing in your free time?    male   

                                            response  
0  Prompt: What do you enjoy doing in your free t...  
1  Prompt: What do you enjoy doing in your free t...  
2  Prompt: What do you enjoy doing in your free t...  
3  Prompt: What do you enjoy doing in your free t...  
4  Prompt: What do you enjoy doing in your free t...  


In [ ]:
from google.colab import files
files.download('gpt2_gendered_responses.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Gender Classification

In [28]:

df = pd.read_csv("gpt2_gendered_responses.csv")
df["UserId"] = df["model"].map({"male": 1, "female": 2})
# Map Gender: male=1, female=0
df["Gender"] = df["model"].map({"male": 1, "female": 0})
df = df.rename(columns={"response": "ReviewText"})
df = df[["UserId", "Gender", "ReviewText"]]
df.to_csv("genderBERT_test.csv", index=False)


In [32]:

df = pd.read_csv('genderBERT_test.csv')
df['ReviewText_clean'] = df['ReviewText'].apply(clean_text)
df['ReviewText'] = df['ReviewText_clean']
df.drop(columns=['ReviewText_clean'], inplace=True)
df.to_csv('genderBERT_test_clean.csv', index=False)


In [35]:

def clean_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'\{.*?\}', ' ', text)
    text = re.sub(r'\(.*?\)', ' ', text)
    text = re.sub(r'[#*\->]+', ' ', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r'(Response:)', '', text, count=1)
    text = re.sub(r'Response:', '', text)
    text = re.sub(r'[\[\]\{\}\(\)\<\>]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df = pd.read_csv('genderBERT_test_clean.csv')

df['ReviewText'] = df['ReviewText'].astype(str).apply(clean_text)
df = df[df['ReviewText'].str.len() > 15]

df.to_csv('genderBERT_test_clean.csv', index=False)

print(df.head(10))


   UserId  Gender                                         ReviewText
0       1       1  Prompt: What do you enjoy doing in your free t...
1       2       0  Prompt: What do you enjoy doing in your free t...
2       1       1  Prompt: What do you enjoy doing in your free t...
3       2       0  Prompt: What do you enjoy doing in your free t...
4       1       1  Prompt: What do you enjoy doing in your free t...
5       2       0  Prompt: What do you enjoy doing in your free t...
6       1       1  Prompt: What do you enjoy doing in your free t...
7       2       0  Prompt: What do you enjoy doing in your free t...
8       1       1  Prompt: What do you enjoy doing in your free t...
9       2       0  Prompt: What do you enjoy doing in your free t...


In [36]:
from google.colab import files
files.download('genderBERT_test_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline

gender_classifier = pipeline(
    "text-classification",
    model="padmajabfrl/Gender-Classification"
)


In [ ]:
predictions = gender_classifier(df["ReviewText"].tolist(), truncation=True)
df["Predicted"] = [p["label"] for p in predictions]


In [ ]:
label_map = {'Female': 0, 'Male': 1}
df["PredictedNumeric"] = df["Predicted"].map(label_map)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

true_labels = df["PredictedNumeric"]
pred_labels = df["Gender"]

accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
conf_matrix = confusion_matrix(true_labels, pred_labels)

# Print results
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")
print("Confusion Matrix:\n", conf_matrix)


In [3]:
df = pd.read_csv('genderBERT_test_clean.csv')
print(df.head())


   UserId  Gender                                         ReviewText
0       1       1  Prompt: What do you enjoy doing in your free t...
1       2       0  Prompt: What do you enjoy doing in your free t...
2       1       1  Prompt: What do you enjoy doing in your free t...
3       2       0  Prompt: What do you enjoy doing in your free t...
4       1       1  Prompt: What do you enjoy doing in your free t...


In [6]:
male_samples = df[df['Gender'] == 1]['ReviewText'].tolist()
female_samples = df[df['Gender'] == 0]['ReviewText'].tolist()


In [8]:
len(male_samples)

150

In [10]:
df = pd.read_csv("genderBERT_test_clean.csv")
def extract_prompt(text):
    match = re.match(r"Prompt: (.*?[\?\.\!])", text)
    return match.group(0) if match else "Prompt: Unknown"

df['prompt'] = df['ReviewText'].apply(extract_prompt)


In [12]:
def extract_response(text):
    # Remove "Prompt: ..." and keep only the answer
    match = re.match(r"Prompt:.*?\? (.+)", text)
    if match:
        return match.group(1).strip()
    return text.strip()

df['response_only'] = df['ReviewText'].apply(extract_response)


In [21]:
def calculate_gender_self_bleu(gender_df, gender_name):
    prompt_groups = gender_df.groupby('prompt')['response_only'].apply(list)
    valid_prompts = prompt_groups[prompt_groups.apply(len) >= 2]

    prompt_bleus = valid_prompts.apply(lambda responses:
        sum(sacrebleu.corpus_bleu([hyp], [responses[:i]+responses[i+1:]],
                                 smooth_method='exp').score
            for i, hyp in enumerate(responses)) / len(responses)
    )
    mean_bleu = prompt_bleus.mean()

    return prompt_bleus, mean_bleu

In [19]:
male_df = df[df['Gender'] == 1]
female_df = df[df['Gender'] == 0]

In [22]:
male_bleus, male_mean = calculate_gender_self_bleu(male_df, "Male")
female_bleus, female_mean = calculate_gender_self_bleu(female_df, "Female")

# Print results
print("Male Model Results:")
print(f"Mean Self-BLEU: {male_mean:.2f}")
print(male_bleus)

print("\nFemale Model Results:")
print(f"Mean Self-BLEU: {female_mean:.2f}")
print(female_bleus)

# Save to CSV
pd.DataFrame({
    'Prompt': male_bleus.index.tolist() + female_bleus.index.tolist(),
    'Self-BLEU': male_bleus.tolist() + female_bleus.tolist(),
    'Gender': ['Male']*len(male_bleus) + ['Female']*len(female_bleus)
}).to_csv("gender_separated_self_bleu.csv", index=False)

Male Model Results:
Mean Self-BLEU: 5.39
prompt
Are you a coffee or tea person?                                    2.106342
Can you describe your perfect weekend?                             1.926690
Do you enjoy cooking or baking?                                   11.001065
Do you have any hobbies or activities you’re passionate about?     3.857542
Do you have any pets?                                             13.857793
Do you like to plan ahead or be spontaneous?                       3.368510
Do you prefer mornings or evenings, and why?                       3.175285
Have you watched any good movies or TV shows lately?              13.420860
How do you like to relax after a long day?                         1.698871
If you could learn any new skill, what would it be?               10.161809
If you could meet any historical figure, who would it be?          3.059058
If you could travel anywhere in the world, where would you go?     5.692821
What do you enjoy doing in your free tim